# Code source du chapitre 3
*mis à jour le 01.07.2022*

## Téléchargement des données

Données utilisées

* [*APUR* - BESOIN THEORIQUE CHAUFFAGE ET TYPOLOGIE AU BATI](https://data-apur.opendata.arcgis.com/datasets/Apur::besoin-theorique-chauffage-et-typologie-au-bati/explore)
* [*IGN* - BD TOPO (Ile de France)](https://geoservices.ign.fr/bdtopo#telechargementshpreg)
* [*IGN* - BD ORTHO (Départements 75, 92, 93 et 94)](https://geoservices.ign.fr/bdortho#telechargement)


In [1]:
from io import BytesIO
from urllib.request import urlopen,urlretrieve
from py7zr import SevenZipFile
import os

# Fonction de téléchargement et d'extraction des jeux de données
def dl_and_extract_zip_from_url(url,out_dir):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    with urlopen(url) as zipresp:
        with SevenZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(out_dir)
    print(f"Extraction de l'archive {url} dans {out_dir} réussie")

def dl_geojson_from_url(url, out_file):
    resp = urlretrieve(url,out_file)
    print(f"Téléchargement de {url} vers {out_file} réussi")

BD_APUR_URL = "https://opendata.arcgis.com/datasets/e1084cb1d0214fab8e998e55f4e1f806_0.geojson"
BD_TOPO_URL = "https://wxs.ign.fr/859x8t863h6a09o9o6fy4v60/telechargement/prepackage/BDTOPOV3-TOUSTHEMES-REGION-PACK_221$bd_topo_3-0_TOUSTHEMES_SHP_LAMB93_R11_2022-03-15/file/BDTOPO_3-0_TOUSTHEMES_SHP_LAMB93_R11_2022-03-15.7z"
BD_ORTHO_URLS = [
    "https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D075_2021$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D075_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D075_2021-01-01.7z",
    "https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D092_2021$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D092_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D092_2021-01-01.7z",
    "https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D093_2021$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D093_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D093_2021-01-01.7z",
    "https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D094_2021-01-01$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D094_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D094_2021-01-01.7z",
]

dl_geojson_from_url(BD_APUR_URL,os.path.join("_DATA","BD_APUR.geojson"))
dl_and_extract_zip_from_url(BD_TOPO_URL,os.path.join("_DATA","BD_TOPO"))
for zip_url in BD_ORTHO_URLS:
    dl_and_extract_zip_from_url(zip_url,os.path.join("_DATA","BD_ORTHO"))

Téléchargement de https://opendata.arcgis.com/datasets/e1084cb1d0214fab8e998e55f4e1f806_0.geojson vers _DATA\BD_APUR.geojson réussie
Extraction de l'archive https://wxs.ign.fr/859x8t863h6a09o9o6fy4v60/telechargement/prepackage/BDTOPOV3-TOUSTHEMES-REGION-PACK_221$BDTOPO_3-0_TOUSTHEMES_SHP_LAMB93_R11_2022-03-15/file/BDTOPO_3-0_TOUSTHEMES_SHP_LAMB93_R11_2022-03-15.7z dans _DATA\BD_TOPO réussie
Extraction de l'archive https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D075_2021$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D075_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D075_2021-01-01.7z dans _DATA\BD_ORTHO réussie
Extraction de l'archive https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargement/prepackage/ORTHOHR-JP2_PACK_D092_2021$ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D092_2021-01-01/file/ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D092_2021-01-01.7z dans _DATA\BD_ORTHO réussie
Extraction de l'archive https://wxs.ign.fr/enlf5fc2u11becs9p951mrzi/telechargeme

## Pré-traitement des données

#### Imports nécessaires

In [1]:
import geopandas as gpd
from shapely import speedups
speedups.enabled

True

#### *APUR* - BESOIN THEORIQUE CHAUFFAGE ET TYPOLOGIE AU BATI

Chargement et lecture

In [2]:
BD_APUR_DATA = os.path.join("_DATA","BD_APUR.geojson")
bd_apur = gpd.read_file(BD_APUR_DATA)
bd_apur_origin_length = len(bd_apur)
# Expression des coordonnées en Lambert 93
bd_apur.crs = 4326
bd_apur = bd_apur.to_crs(2154)

bd_apur.head(5)

,OBJECTID,N_SQ_EB,C_CAINSEE,C_PERCONST_APUR,C_PERCONST_INSEE,B_DALLE,DUR,DUR_CODE,H_MEDIANE,N_SQ_PC,...,GV,BV,N_INERTIE,BESOINS_AVT_INTER,INTERMITTENCE,BESOINS_POST_INTER,RATIO_BESOINS_POST_INTER,Shape_Length,Shape_Area,geometry
0,1,750070992.0,75119.0,3,1.0,N,None,None,8.879,750069354.0,...,632.440186,597.702680,3.6,37655.268848,0.741555,27923.458071,321.773863,32.451442,34.495045,"POLYGON ((654091.003 6866170.532, 654088.744 6..."
1,2,750016084.0,75116.0,3,1.0,N,None,None,20.671,750049707.0,...,3788.432068,3412.838303,3.6,215008.813075,0.889257,191198.111495,194.231236,58.024597,168.075870,"POLYGON ((646688.801 6862086.526, 646690.934 6..."
2,3,750069338.0,75114.0,1,1.0,N,None,None,3.297,750037736.0,...,924.635739,864.387801,3.6,54456.431450,0.595141,32409.230536,203.480189,64.185734,170.502193,"POLYGON ((650449.151 6859688.261, 650443.451 6..."
3,4,750093751.0,75117.0,3,1.0,N,None,None,18.792,750055762.0,...,2964.148488,2549.677502,3.6,160629.682606,0.901857,144864.969869,130.107949,83.576085,209.116777,"POLYGON ((650449.828 6865554.929, 650442.522 6..."
4,5,750042312.0,75115.0,11,6.0,N,None,None,17.011,750044851.0,...,2643.346746,2149.725254,2.1,135432.690996,0.913685,123742.781515,80.427171,79.313684,319.219614,"POLYGON ((648436.487 6861150.939, 648430.955 6..."


Encodage

In [3]:
# Encodage de la destination du bâtiment
NATURE = {
    "logement individuel" : 0,
    "logement collectif" : 1,
    "bâtiment mixte" : 2,
    "batiment tertiaire ou industriel" : 3,
    "non déterminée" : 4 
    }
# Application de la table d'encodage
bd_apur["Typologie_apur"] = bd_apur["Typologie_apur"].map(NATURE)
bd_apur["Typologie_apur"] = bd_apur["Typologie_apur"].astype('Int64')

Sélection des champs utiles

In [4]:
# Filtrage des bâtiments avec un nombre de niveaux supérieur à 0
# et une période de construction n'étant pas indéfinie (code : 99)
bd_apur = bd_apur[(bd_apur["NB_NIV"] > 0) & (bd_apur["C_PERCONST_APUR"] != 99)]

# Table de rennomage des champs utiles
APUR_FIELDS = {
    "SURFACE_VITRAGE" : "surface_vitrage",
    "SURFACE_HABITABLE" : "surface_habitable",
    "C_PERCONST_APUR" : "periode_construction",
    "Shape_Area" : "surface_emprise",
    "SURFACE_PAROI_TOT" : "surface_parois",
    "NB_NIV" : "nb_niveaux",
    "H_MEDIANE" : "hauteur_mediane",
    "Typologie_apur" : "typologie_bati_apur"
}
# Sélection et renommage des champs utiles
bd_apur = bd_apur[["geometry"] + list(APUR_FIELDS.keys())]
bd_apur = bd_apur.rename(columns=APUR_FIELDS)
# Suppression des bâtiments avec au moins un des champs manquants
bd_apur = bd_apur.dropna()

print(f"Nombre de bâtiments :\n\tavant traitement : {bd_apur_origin_length}\n\taprès traitement : {len(bd_apur)}")
bd_apur.head(5)

Nombre de bâtiments :
	avant traitement : 1113344
	après traitement : 901225


,geometry,surface_vitrage,surface_habitable,periode_construction,surface_emprise,surface_parois,nb_niveaux,hauteur_mediane,typologie_bati_apur
0,"POLYGON ((654091.003 6866170.532, 654088.744 6...",55.867366,86.779758,3,34.495045,288.136350,2.959667,8.879,1
1,"POLYGON ((646688.801 6862086.526, 646690.934 6...",323.876300,984.383953,3,168.075870,1199.426441,6.890333,20.671,1
2,"POLYGON ((650449.151 6859688.261, 650443.451 6...",41.745613,159.274624,1,170.502193,211.620366,1.099000,3.297,1
3,"POLYGON ((650449.828 6865554.929, 650442.522 6...",207.610361,1113.421365,3,209.116777,1570.561787,6.264000,18.792,1
4,"POLYGON ((648436.487 6861150.939, 648430.955 6...",426.507516,1538.569376,11,319.219614,1349.205078,5.670333,17.011,2


#### *IGN* - BD TOPO (Ile de France)

Chargement et lecture

In [5]:
BD_TOPO_DATA = os.path.join(
    "_DATA",
    "BD_TOPO",
    "BDTOPO_3-0_TOUSTHEMES_SHP_LAMB93_R11_2022-03-15", 
    "BDTOPO", 
    "1_DONNEES_LIVRAISON_2022-03-00084", 
    "BDT_3-0_SHP_LAMB93_R11-ED2022-03-15", 
    "BATI", 
    "BATIMENT.shp"
    )

bd_topo = gpd.read_file(BD_TOPO_DATA)
bd_topo_origin_length = len(bd_topo)
bd_topo.head(5)

,ID,NATURE,USAGE1,USAGE2,LEGER,ETAT,DATE_CREAT,DATE_MAJ,DATE_APP,DATE_CONF,...,MAT_MURS,MAT_TOITS,HAUTEUR,Z_MIN_SOL,Z_MIN_TOIT,Z_MAX_TOIT,Z_MAX_SOL,ORIGIN_BAT,APP_FF,geometry
0,BATIMENT0000000338465630,"Industriel, agricole ou commercial",Agricole,None,Non,En service,2014-10-02 18:42:00,2017-02-27 12:32:38,None,None,...,None,None,4.9,93.7,98.6,NaN,NaN,Cadastre,None,"POLYGON Z ((682392.800 6839868.300 98.600, 682..."
1,BATIMENT0000000338462974,"Industriel, agricole ou commercial",Indifférencié,None,Non,En service,2014-10-02 18:40:15,2017-02-27 12:32:38,None,None,...,None,None,2.2,99.0,101.2,NaN,NaN,Cadastre,None,"POLYGON Z ((683189.200 6840307.900 101.200, 68..."
2,BATIMENT0000000338462973,"Industriel, agricole ou commercial",Indifférencié,None,Oui,En service,2014-10-02 18:40:15,2017-02-27 12:32:38,None,None,...,None,None,2.5,99.0,101.5,NaN,NaN,Cadastre,None,"POLYGON Z ((683198.500 6840313.300 101.500, 68..."
3,BATIMENT0000000338462982,"Industriel, agricole ou commercial",Indifférencié,None,Oui,En service,2014-10-02 18:40:15,2017-02-27 12:32:38,None,None,...,None,None,4.4,97.3,101.7,NaN,NaN,Cadastre,None,"POLYGON Z ((683178.000 6840290.600 101.700, 68..."
4,BATIMENT0000000338462983,"Industriel, agricole ou commercial",Indifférencié,None,Non,En service,2014-10-02 18:40:15,2017-02-27 12:32:38,None,None,...,None,None,4.6,97.1,101.7,NaN,NaN,Cadastre,None,"POLYGON Z ((683184.200 6840286.400 101.700, 68..."


Encodage

In [6]:
# Encondage des valeurs de matériaux
# D'après la table de correspondance des métadonnées de la BD TOPO
WALL_MATERIAL = {
    1 : ["1","01","10","11","19","91"], # Pierre
    2 : ["2","02","12","20","21","22","29","92"], # Meulière
    3 : ["3","03","13","23","30","31","32","33","34","36","39","43","63","93"], # Béton
    4 : ["4","04","14","24","40","41","42","44","49","94"], # Briques
    5 : ["5","05","15","25","35","45","50","51","52","53","54","55","56","59","65","95"], # Aggloméré
    6 : ["6","06","16","26","46","60","61","62","64","66","96"] # Bois
}

ROOF_MATERIAL = {
    1 : ["1","01","10","11","13","19","31","91"], # Tuiles
    2 : ["2","02","12","20","21","22","23","24","29","32","42","92"], # Ardoises
    3 : ["3","03","30","33","39","93"], # Zinc/Aluminium/Tôle
    4 : ["4","04","14","34","40","41","43","44","49","94"] # Béton
}

def encode_materials(mat, D):
    for key,values in D.items():
        if mat in values: 
            return key
    return 0

bd_topo["MAT_MURS"] = bd_topo["MAT_MURS"].apply(encode_materials, D=WALL_MATERIAL)
bd_topo["MAT_MURS"] = bd_topo["MAT_MURS"].astype('Int64')
bd_topo["MAT_TOITS"] = bd_topo["MAT_TOITS"].apply(encode_materials, D=ROOF_MATERIAL)
bd_topo["MAT_TOITS"] = bd_topo["MAT_TOITS"].astype('Int64')

Sélection des champs utiles

In [7]:
# Filtrage des bâtiments en "dur" uniquement
# avec un matériau de toiture n'étant pas "indéfini"
bd_topo = bd_topo[
    (bd_topo["LEGER"] == "Non") &
    (bd_topo["MAT_MURS"] != 0) & 
    (bd_topo["MAT_TOITS"] != 0)
    ]

# Table de rennomage des champs utiles
BD_TOPO_FIELDS = {
    "MAT_MURS" : "materiau_facade",
    "MAT_TOITS" : "materiau_toiture"
}

# Sélection et renommage des champs utiles
bd_topo = bd_topo[["geometry"] + list(BD_TOPO_FIELDS.keys())]
bd_topo = bd_topo.rename(columns=BD_TOPO_FIELDS)
# Suppression des bâtiments avec au moins un des champs manquants
bd_topo = bd_topo.dropna()

print(f"Nombre de bâtiments :\n\tavant traitement : {bd_topo_origin_length}\n\taprès traitement : {len(bd_topo)}")
bd_topo.head(5)

Nombre de bâtiments :
	avant traitement : 3300355
	après traitement : 1423140


,geometry,materiau_facade,materiau_toiture
8,"POLYGON Z ((661749.900 6857704.800 43.200, 661...",5,4
27,"POLYGON Z ((661934.400 6857541.100 46.100, 661...",5,1
56,"POLYGON Z ((661943.200 6857567.700 42.700, 661...",4,1
58,"POLYGON Z ((661767.500 6857511.100 42.200, 661...",4,1
65,"POLYGON Z ((657761.900 6795304.300 86.300, 657...",5,1


## Agrégation

BESOIN THEORIQUE CHAUFFAGE ET TYPOLOGIE AU BATI et BD TOPO

In [8]:
# Création d'un polygone de taille réduite 
# au niveau des centroide des bâtiments de la BD TOPO
# afin de permettre le test d'inclusion
bd_topo["geometry"] = bd_topo["geometry"].apply(lambda poly : poly.centroid.buffer(0.1))

# Recoupement géométrique entre les centroides 
# des bâtiments de la BD TOPO et les emprises
# des bâtiments de l'APUR
intersection = gpd.sjoin(bd_apur,bd_topo, how='inner')
intersection = intersection.drop(columns=["index_right"])
intersection['id'] = intersection.index

data_agg_path = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS.gpkg")
intersection.to_file(data_agg_path, driver='GPKG', layer='bati')

print(f"Nombre de bâtiments après agrégation : {len(intersection)}")
intersection.head(5)

Nombre de bâtiments après agrégation : 434376


,geometry,surface_vitrage,surface_habitable,periode_construction,surface_emprise,surface_parois,nb_niveaux,hauteur_mediane,typologie_bati_apur,materiau_facade,materiau_toiture,id
1,"POLYGON ((646688.801 6862086.526, 646690.934 6...",323.876300,984.383953,3,168.075870,1199.426441,6.890333,20.671,1,2,3,1
3,"POLYGON ((650449.828 6865554.929, 650442.522 6...",207.610361,1113.421365,3,209.116777,1570.561787,6.264000,18.792,1,5,3,3
7,"POLYGON ((650123.622 6858949.072, 650117.173 6...",206.804731,835.155769,3,127.806816,1271.248075,7.687667,23.063,1,1,2,7
9,"POLYGON ((649246.766 6860663.041, 649245.063 6...",323.496695,1574.377016,5,200.643633,1822.730079,9.231333,27.694,1,4,4,9
16,"POLYGON ((649774.818 6863668.145, 649793.830 6...",329.506195,1117.689973,3,191.745887,1607.349513,6.857667,20.573,2,1,2,16


*IGN* - BD ORTHO (Départements 75, 92, 93 et 94)

In [5]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

DATA_AGG_PATH = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS.gpkg")
IMG_H, IMG_W = 60, 60
BUILDINGS_IMGS_DIR = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS_imgs")
PROCESSED_IDS = []
BD_ORTHO_DIRS = [
    os.path.join("_DATA", "BD_ORTHO", "ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D075_2021-01-01", "ORTHOHR", "1_DONNEES_LIVRAISON_2022-06-00123", "OHR_RVB_0M20_JP2-E080_LAMB93_D75-2021"),
    os.path.join("_DATA", "BD_ORTHO", "ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D092_2021-01-01", "ORTHOHR", "1_DONNEES_LIVRAISON_2022-06-00141", "OHR_RVB_0M20_JP2-E080_LAMB93_D92-2021"),
    os.path.join("_DATA", "BD_ORTHO", "ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D093_2021-01-01", "ORTHOHR", "1_DONNEES_LIVRAISON_2022-06-00256", "OHR_RVB_0M20_JP2-E080_LAMB93_D93-2021"),
    os.path.join("_DATA", "BD_ORTHO", "ORTHOHR_1-0_RVB-0M20_JP2-E080_LAMB93_D094_2021-01-01", "ORTHOHR", "1_DONNEES_LIVRAISON_2022-06-00139", "OHR_RVB_0M20_JP2-E080_LAMB93_D94-2021")
]

def extract_building_aerial_image(building, tile):
    building_id = building["id"]
    img_path = os.path.join(BUILDINGS_IMGS_DIR,f"{building_id}.png")
    building_polygon = building["geometry"].buffer(0.1,cap_style=3,join_style=2)
    # Découpage de l'image aérienne grâce au polygone d'emprise du bâtiment
    if not os.path.exists(img_path):
        img,trs = mask(tile,[building_polygon],crop=True,pad=True)
        with rasterio.open(img_path,"w+",transform=trs,dtype=rasterio.uint8,driver="PNG",height=IMG_H,width=IMG_W,count=3) as p:
            p.write(img)
            
    PROCESSED_IDS.append(building_id)
    print(f"""Extraction de la photographie aérienne du bâtiment {len(PROCESSED_IDS)} (id : {building_id})""",end="\r")

# Itération à travers chaque image aérienne (carreau) de chaque département
for bd_ortho_dir in BD_ORTHO_DIRS:
    for file in os.listdir(bd_ortho_dir):
        if file.endswith(".jp2"):
            tile = rasterio.open(os.path.join(bd_ortho_dir,file))
            tile_bounds = tile.bounds
            # Sélection des bâtiments de la base agrégée situés sur le carreau chargé
            tile_buildings = gpd.read_file(DATA_AGG_PATH,bbox=tile_bounds)
            # Filtrage des bâtiments déjà traités (car à cheval sur d'autres carreaux)
            tile_buildings = tile_buildings[~tile_buildings["id"].isin(PROCESSED_IDS)]
            if not tile_buildings.empty:
                # Extraction et enregistrement des images aériennes des bâtiments sélectionnés
                tile_buildings = tile_buildings.apply(extract_building_aerial_image, axis=1, tile=tile)
            

#### SYNTHESE

Total de bâtiments :

||APUR|IGN|
|---|---|---|
|Avant prétraitement|1113344|3300355|
|Après prétraitement|901225|1423140|

Total après agrégation : **434 376**

In [7]:
from IPython.core.display import HTML
# Affichage de quelques bâtiments après agrégation
sample_buildings = gpd.read_file(DATA_AGG_PATH,rows=5)
sample_buildings = sample_buildings.drop(columns="geometry")
sample_buildings["img"] = sample_buildings["id"].apply(lambda x: '<img src="' + os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS_imgs",f"{x}.png") + '" width="60">')

# Intégration des images dans la table
HTML(sample_buildings.to_html(escape=False))

,surface_vitrage,surface_habitable,periode_construction,surface_emprise,surface_parois,nb_niveaux,hauteur_mediane,typologie_bati_apur,materiau_facade,materiau_toiture,id,img
0,323.876300,984.383953,3,168.075870,1199.426441,6.890333,20.671,1,2,3,1,
1,207.610361,1113.421365,3,209.116777,1570.561787,6.264000,18.792,1,5,3,3,
2,206.804731,835.155769,3,127.806816,1271.248075,7.687667,23.063,1,1,2,7,
3,323.496695,1574.377016,5,200.643633,1822.730079,9.231333,27.694,1,4,4,9,
4,329.506195,1117.689973,3,191.745887,1607.349513,6.857667,20.573,2,1,2,16,


#### LABELS

<table align="top">
<tr style="vertical-align:top"><th>periode_construction</th><th>typologie_bati_apur</th><th>materiau_facade</th><th>materiau_toiture</th></tr>
<tr style="vertical-align:top"><td>

|*Code*|*Label*           |
|---|---|
|1 | Avant 1800           |
|2 | 1801-1850            |
|3 | 1851-1914            |
|4 | 1915-1939            |
|5 | 1940-1967            |
|6 | 1968-1975            |
|7 | 1976-1981            |
|8 | 1982-1989            |
|9 | 1990-1999            |
|10| 2000-2007            |
|11| Après 2008           |

</td><td>

|*Code*|*Label*                     |
|---|---|
|0| logement individuel             |
|1| logement collectif              |
|2| bâtiment mixte                  |
|3| batiment tertiaire ou industriel|
|4| non déterminée                  |
 
</td><td>

|*Code*|*Label*     |
|---|---|
|1| Pierre          |
|2|Meulière         |
|3|Béton            |
|4|Briques          |
|5|Aggloméré        |
|6|Bois             |

</td><td>

|*Code*|*Label*|
|---|---|        
|1|Tuiles|
|2|Ardoises|
|3|Zinc/Aluminium/Tôle|
|4|Béton|

</td></tr> </table>

## Étude des corrélations

In [9]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

DATA_AGG_PATH = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS.gpkg")

# Chargement du jeu agrégé
buildings = gpd.read_file(DATA_AGG_PATH)

# Déclinaison des colonnes catégoriques en colonnes indicatrices pour chaque catégorie possible
# _______| typologie |   --->   _______| typologie_a | typologie_b |
# bati_1 | a         |   --->   bati_1 | 1           | 0           |
# bati_2 | b         |   --->   bati_2 | 0           | 1           |
cols_to_decline = ["periode_construction","typologie_bati_apur","materiau_facade","materiau_toiture"]
buildings[cols_to_decline] = buildings[cols_to_decline].astype("category")
buildings = pd.get_dummies(buildings,prefix=cols_to_decline)
buildings = buildings.drop(columns=["id","geometry"])
# Calcul des coefficients de corrélation
correlation = buildings.corr()
# Construction et export du corrélogramme global
fig = px.imshow(correlation, color_continuous_scale='RdBu_r')
fig.write_html(os.path.join("_OUTPUT", "matrice_corrélation.html"))

# Orientation du corrélogramme selon la destination du bâtiment
DESTINATIONS = {
    "typologie_bati_apur_2" : "bâtiment mixte",
    "typologie_bati_apur_1" : "logement collectif",
    "typologie_bati_apur_0" : "logement individuel"
}
correl_destination = correlation.loc[list(DESTINATIONS.keys())]
correl_destination = correl_destination.rename(index=DESTINATIONS)
# Construction et export du corrélogramme par destination du bâtiment
fig2 = px.imshow(correl_destination, color_continuous_scale='RdBu_r')
fig2.write_html(os.path.join("_OUTPUT", "matrice_corrélation_par_destination.html"))

fig2

## Entraînement d'un réseau de neurones

#### Exemple : Classification de matériaux de toiture à partir de photographies aériennes

In [4]:
import geopandas as gpd
import tensorflow as tf
import autokeras as ak
import numpy as np
from PIL import Image

DATA_AGG_PATH = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS.gpkg")
BUILDINGS_IMGS_DIR = os.path.join("_DATA_agg","BATIMENTS_GRAND_PARIS_imgs")

def load_img(building_id):
    path = os.path.join(BUILDINGS_IMGS_DIR,f"{building_id}.png")
    if os.path.exists(path):
        img = Image.open(path)
        return np.asarray(img)
    return None

buildings = gpd.read_file(DATA_AGG_PATH,rows=100000) # Essai avec 100 000 bâtiments dans ce notebook
# Chargement des photographies et suppression des bâtiments sans photographie
buildings["img"] = buildings["id"].apply(load_img)
buildings = buildings.dropna()
# Sélection des données pour l'entraînement
# X : images sur lesquelles se basent la prédiction
X = np.stack(buildings["img"].to_list())
# y : "labels" correspondants
y = buildings["materiau_toiture"].to_numpy()

# Dimensions de X et dimension de chaque image (3 valeurs par pixel sur 60x60 pixels)
print(X.shape, X[0].shape)
# Dimensions de y et dimension de chaque label (une seule valeur numérique)
print(y.shape, y[0].shape)

(100000, 60, 60, 3) (60, 60, 3)
(100000,) ()


In [5]:
# Initialisation du modèle
clf = ak.ImageClassifier(metrics=['accuracy'], max_trials=1, directory=os.path.join("_MDL","mdl_prediction_mat_toiture_temp"))
# Entraînement
clf.fit(X, y, validation_split=0.15, epochs=10)
# Sauvegarde du modèle
mdl_path = os.path.join("_MDL","mdl_prediction_mat_toiture")
mdl = clf.export_model()
mdl.save(mdl_path, save_format="tf")

INFO:tensorflow:Reloading Oracle from existing project _MDL\mdl_prediction_mat_toiture_temp\image_classifier\oracle.json


INFO:tensorflow:Reloading Oracle from existing project _MDL\mdl_prediction_mat_toiture_temp\image_classifier\oracle.json


INFO:tensorflow:Reloading Tuner from _MDL\mdl_prediction_mat_toiture_temp\image_classifier\tuner0.json


INFO:tensorflow:Reloading Tuner from _MDL\mdl_prediction_mat_toiture_temp\image_classifier\tuner0.json


INFO:tensorflow:Oracle triggered exit


...

Epoch 1/10
3125/3125 [==============================] - 427s 136ms/step - loss: 0.7138 - accuracy: 0.7258
Epoch 2/10
3125/3125 [==============================] - 417s 134ms/step - loss: 0.6764 - accuracy: 0.7415
Epoch 3/10
3125/3125 [==============================] - 449s 144ms/step - loss: 0.6636 - accuracy: 0.7456
Epoch 4/10
3125/3125 [==============================] - 452s 145ms/step - loss: 0.6533 - accuracy: 0.7522
Epoch 5/10
3125/3125 [==============================] - 436s 140ms/step - loss: 0.6390 - accuracy: 0.7577
Epoch 6/10
3125/3125 [==============================] - 429s 137ms/step - loss: 0.6285 - accuracy: 0.7621
Epoch 7/10
3125/3125 [==============================] - 429s 137ms/step - loss: 0.6199 - accuracy: 0.7638
Epoch 8/10
3125/3125 [==============================] - 432s 138ms/step - loss: 0.6153 - accuracy: 0.7656
Epoch 9/10
3125/3125 [==============================] - 431s 138ms/step - loss: 0.6056 - accuracy: 0.7706
Epoch 10/10
3125/3125 [=======================

INFO:tensorflow:Assets written to: _MDL\mdl_prediction_mat_toiture_temp\image_classifier\best_model\assets


INFO:tensorflow:Assets written to: _MDL\mdl_prediction_mat_toiture_temp\image_classifier\best_model\assets


INFO:tensorflow:Assets written to: _MDL\mdl_prediction_mat_toiture\assets


INFO:tensorflow:Assets written to: _MDL\mdl_prediction_mat_toiture\assets


#### Prédiction sur de nouvelles images

<table align="top">
<tr style="vertical-align:top"><th>img1</th><th>img2</th><th>img3</th><th>img4</th></tr>
<tr style="vertical-align:top"><td>
<img src="_DATA_test/img1.png">
</td><td>
<img src="_DATA_test/img2.png">
</td><td>
<img src="_DATA_test/img3.png">
</td><td>
<img src="_DATA_test/img4.png">
</td></tr> </table>

In [7]:
from tensorflow.keras.models import load_model

X_TEST_NAMES = ["img1.png","img2.png","img3.png","img4.png"]
X_TEST_DIR = "_DATA_test"
MDL_PATH = os.path.join("_MDL","mdl_prediction_mat_toiture")

# Chargement des images de test
X_test = np.stack([np.asarray(Image.open(os.path.join(X_TEST_DIR,test)).convert("RGB")) for test in X_TEST_NAMES])

# Chargement du modèle et prédiction
mdl = load_model(MDL_PATH, custom_objects=ak.CUSTOM_OBJECTS)
y_test = mdl.predict(X_test)

# Rappel
MATERIALS = {
    1 :"Tuiles",
    2 :"Ardoises",
    3 :"Zinc/Aluminium/Tôle",
    4 :"Béton"
}

print("Résultats du modèle (entraîné sur 100 000 bâtiments)")

# Résultats
for name,y in zip(X_TEST_NAMES,y_test):
    # Obtention de la catégorie correspondant au pourcentage le plus élevé (catégorie la plus "probable")
    max_pred = max(y)
    max_index = list(y).index(max_pred)
    print(f"{name} : matériau {MATERIALS[max_index+1]}")

1/1 [==============================] - 0s 50ms/step
Résultats du modèle (entraîné sur 100 000 bâtiments)
img1.png : matériau Tuiles
img2.png : matériau Béton
img3.png : matériau Zinc/Aluminium/Tôle
img4.png : matériau Tuiles
